In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, product

In [2]:
df = pd.read_csv('../../abc/full_dts.csv')

C:\Users\admin\AppData\Local\Temp\ipykernel_18632\3766361288.py:1: DtypeWarning: Columns (25,47,55,57,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../abc/full_dts.csv')


In [3]:
df.shape

(2616579, 85)

In [4]:
df[' Label'].value_counts()

 Label
BENIGN                       1210210
ADWARE_GOOLIGAN                93772
ADWARE_FEIWO                   56632
SCAREWARE_ANDROIDDEFENDER      56440
RANSOMWARE_SVPENG              54161
RANSOMWARE_PORNDROID           46082
SCAREWARE_FAKEAPPAL            44563
RANSOMWARE_KOLER               44555
SMSMALWARE_NANDROBOX           44517
ADWARE_EWIND                   43374
SCAREWARE_AVFORANDROID         42448
SCAREWARE_AVPASS               40776
SCAREWARE_FAKEAV               40089
RANSOMWARE_RANSOMBO            39859
SMSMALWARE_PLANKTON            39765
ADWARE_DOWGIN                  39682
RANSOMWARE_CHARGER             39551
ADWARE_SHUANET                 39271
ADWARE_KEMOGE                  38771
RANSOMWARE_SIMPLOCKER          36340
ADWARE_YOUMI                   36035
SCAREWARE_FAKEAPP              34676
SMSMALWARE_BIIGE               33678
SMSMALWARE_SMSSNIFFER          33618
SCAREWARE_FAKETAOBAO           33299
RANSOMWARE_WANNALOCKER         32701
ADWARE_KOODOUS                 

In [5]:
df_cp = df.copy()

In [6]:
def clean_df(df):
    # Remove the space before each feature names
    df.columns = df.columns.str.strip()
    print('dataset shape', df.shape)

    # This set of feature should have >= 0 values
    num = df._get_numeric_data()
    num[num < 0] = 0

    zero_variance_cols = []
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(zero_variance_cols, axis = 1, inplace = True)
    print('zero variance columns', zero_variance_cols, 'dropped')
    print('shape after removing zero variance columns:', df.shape)

    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    print(df.isna().any(axis = 1).sum(), 'rows dropped')
    df.dropna(inplace = True)
    print('shape after removing nan:', df.shape)

    # Drop duplicate rows
    df.drop_duplicates(inplace = True)
    print('shape after dropping duplicates:', df.shape)

    column_pairs = [(i, j) for i, j in combinations(df, 2) if df[i].equals(df[j])]
    ide_cols = []
    for column_pair in column_pairs:
        ide_cols.append(column_pair[1])
    df.drop(ide_cols, axis = 1, inplace = True)
    print('columns which have identical values', column_pairs, 'dropped')
    print('shape after removing identical value columns:', df.shape)
    return df
df_cp = clean_df(df_cp)

dataset shape (2616579, 85)
zero variance columns [] dropped
shape after removing zero variance columns: (2616579, 85)
13 rows dropped
shape after removing nan: (2616566, 85)
shape after dropping duplicates: (2616566, 85)
columns which have identical values [('Total Fwd Packets', 'Subflow Fwd Packets'), ('Total Backward Packets', 'Subflow Bwd Packets'), ('Total Length of Fwd Packets', 'Subflow Fwd Bytes'), ('Total Length of Bwd Packets', 'Subflow Bwd Bytes'), ('Fwd PSH Flags', 'SYN Flag Count'), ('Bwd PSH Flags', 'Fwd URG Flags'), ('Bwd PSH Flags', 'Bwd URG Flags'), ('Bwd PSH Flags', 'RST Flag Count'), ('Bwd PSH Flags', 'ECE Flag Count'), ('Bwd PSH Flags', 'Fwd Avg Packets/Bulk'), ('Bwd PSH Flags', 'Fwd Avg Bulk Rate'), ('Bwd PSH Flags', 'Bwd Avg Bytes/Bulk'), ('Bwd PSH Flags', 'Bwd Avg Packets/Bulk'), ('Bwd PSH Flags', 'Bwd Avg Bulk Rate'), ('Fwd URG Flags', 'Bwd URG Flags'), ('Fwd URG Flags', 'RST Flag Count'), ('Fwd URG Flags', 'ECE Flag Count'), ('Fwd URG Flags', 'Fwd Avg Packets/B

In [7]:
df_cpz = df_cp.copy()

In [8]:
df_cp = df_cp.loc[(df_cp['Source Port'] == 443) | (df_cp['Destination Port'] == 443)]

In [9]:
# df_cp = df_cpz.copy()

In [10]:
df_cp.loc[df_cp.Label != 'BENIGN', 'label'] = 1
df_cp.loc[df_cp.Label == 'BENIGN', 'label'] = 0

In [11]:
drop_columns = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'Protocol', 'Label']
df_cp = df_cp.drop(drop_columns, axis=1)

In [12]:
df_cp.shape

(1224657, 63)

In [13]:
for i in df_cp.columns:
    df_cp[i] = df_cp[i].astype('float')

In [14]:
df_cp['label'].value_counts()

label
1.0    642487
0.0    582170
Name: count, dtype: int64

In [15]:
df_cp.columns

Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'PSH Flag Count',
       'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', '

In [16]:
df_cp.loc[(df_cp['Total Length of Fwd Packets'] == 0) & (df_cp['Total Length of Bwd Packets'] == 0)].shape

(543610, 63)

In [18]:
df_cp.shape

(1224657, 63)

In [37]:
from sklearn.feature_selection import SelectKBest


def select_k_best(score, X, Y):
    selector = SelectKBest(score, k=20)
    selector.fit_transform(X, Y)
    names = X.columns.values[selector.get_support()]
    scores = selector.scores_[selector.get_support()]
    names_scores = list(zip(names, scores))
    df_reduced = pd.DataFrame(data=names_scores, columns=['feature_names', 'score'])

    df_reduced = df_reduced.sort_values(['score', 'feature_names'], ascending=[False, True])
    print(df_reduced)
    return df_reduced.feature_names

In [ ]:
from sklearn.feature_selection import mutual_info_classif, f_classif

frs_mi = select_k_best(mutual_info_classif, df_cp[df_cp.columns.difference(['Label'])], df_cp.Label)

In [ ]:
frs_anova = select_k_best(f_classif, df_cp[df_cp.columns.difference(['Label'])], df_cp.Label)

In [10]:
correlation =df_cp.corr()
correlation['Label'].sort_values(ascending = False)

Label                     1.000000
Fwd IAT Total             0.033909
Init_Win_bytes_forward    0.029300
Fwd IAT Max               0.028861
PSH Flag Count            0.025404
                            ...   
ACK Flag Count           -0.040995
URG Flag Count           -0.043195
Bwd PSH Flags                  NaN
CWE Flag Count                 NaN
Fwd Avg Bytes/Bulk             NaN
Name: Label, Length: 63, dtype: float64

In [11]:
drop_columns = ['Bwd PSH Flags', 'CWE Flag Count', 'Fwd Avg Bytes/Bulk']
df_cp = df_cp.drop(drop_columns, axis=1)

In [17]:
corrlabel = correlation['Label'].sort_values(ascending = False)
features = corrlabel[corrlabel >= 0.02].index
features

Index(['Label', 'Fwd IAT Total', 'Init_Win_bytes_forward', 'Fwd IAT Max',
       'PSH Flag Count', 'Fwd IAT Std', 'Flow Duration', 'Min Packet Length',
       'Active Mean', 'Active Max', 'Active Min'],
      dtype='object')

In [17]:
df_cp.columns

Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'PSH Flag Count',
       'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', '

In [18]:
features = ['Flow Duration', 'Flow IAT Max', 'Flow Packets/s', 'Flow IAT Mean', 'Fwd Packets/s', 'Bwd Packets/s']

In [20]:
features = ['Flow Duration', 'Flow IAT Max', 'Flow Packets/s', 'Flow IAT Mean', 'Fwd Packets/s', 'Bwd Packets/s']
X = pd.get_dummies(df_cp[features])
# X = df_test.drop(['Label'], axis=1)
y = df_cp.label

In [27]:
ftpos = []
for ft in X.columns:
    if X.loc[X[ft] < 0].size > 0:
        ftpos.append(ft)
        print(ft)
        print(X.loc[X[ft] < 0].shape[0])

In [22]:
X.shape

(1224657, 6)

Scaler

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

Test Ml

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, stratify = y, random_state = 0, test_size = 0.1)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from xgboost import XGBClassifier


model = RandomForestClassifier(random_state=27)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=27)

In [26]:
from sklearn import metrics

preds_val = model.predict(X_train)
classification = metrics.classification_report(y_train, preds_val)
confusion_matrix = metrics.confusion_matrix(y_train, preds_val)

In [27]:
print(accuracy_score(y_train, preds_val))
print("Confusion matrix:" "\n", confusion_matrix)
print("Classification report:" "\n", classification) 

0.8856151066375973
Confusion matrix:
 [[440982  82971]
 [ 43103 535135]]
Classification report:
               precision    recall  f1-score   support

         0.0       0.91      0.84      0.87    523953
         1.0       0.87      0.93      0.89    578238

    accuracy                           0.89   1102191
   macro avg       0.89      0.88      0.88   1102191
weighted avg       0.89      0.89      0.89   1102191



In [28]:
from sklearn import metrics

preds_val = model.predict(X_test)
classification = metrics.classification_report(y_test, preds_val)
confusion_matrix = metrics.confusion_matrix(y_test, preds_val)

In [29]:
print(accuracy_score(y_test, preds_val))
print("Confusion matrix:" "\n", confusion_matrix)
print("Classification report:" "\n", classification) 

0.6278069015073572
Confusion matrix:
 [[31697 26520]
 [19061 45188]]
Classification report:
               precision    recall  f1-score   support

         0.0       0.62      0.54      0.58     58217
         1.0       0.63      0.70      0.66     64249

    accuracy                           0.63    122466
   macro avg       0.63      0.62      0.62    122466
weighted avg       0.63      0.63      0.63    122466

